In [1]:
import pandas as pd
import re
from textblob import TextBlob
from unidecode import unidecode
import emoji
from nltk.corpus import stopwords
import nltk

# Certifique-se de ter baixado os recursos do NLTK
nltk.download('stopwords')

# Carrega o dataset
df = pd.read_csv("dataset_zap_2.csv")  # ou dataset_zap_2.csv

# Remove linhas vazias
df = df.dropna(subset=["text_content_anonymous"])

# Remove duplicatas
df = df.drop_duplicates(subset=["text_content_anonymous"])

# Lista de stopwords extendida
stop_words = set(stopwords.words('portuguese'))
stop_words.update([
    "bom dia", "boa noite", "boa tarde", "amém", "amem", "deus é fiel", ".", ",", "👍", "🙏", "🔥", "👏", "💪",
    "kkk", "kkkk", "sim", "não", "grato", "grata", "obrigado", "obrigada", "valeu", "ok", "tá", "eh", "ah", "oie"
])

# Função de pré-processamento
def preprocessar(texto):
    if pd.isna(texto):
        return ""
    # Remove emojis
    texto = emoji.replace_emoji(texto, replace='')
    # Remove caracteres especiais e números
    texto = re.sub(r"[^a-zA-ZÀ-ÿ\s]", "", texto)
    # Remove acentuação
    texto = unidecode(texto)
    # Minúsculas
    texto = texto.lower()
    # Remove stopwords personalizadas
    for sw in stop_words:
        texto = texto.replace(sw, "")
    # Remove espaços múltiplos
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

# Aplica pré-processamento
df["texto_limpo"] = df["text_content_anonymous"].apply(preprocessar)

# Remove mensagens que ficaram vazias
df = df[df["texto_limpo"].str.strip().astype(bool)]

# Calcula polaridade com TextBlob
df["score_sentiment"] = df["texto_limpo"].apply(lambda x: TextBlob(x).sentiment.polarity)

# Seleciona mensagens mais positivas e negativas
top_positivas = df.sort_values(by="score_sentiment", ascending=False).head(10)
top_negativas = df.sort_values(by="score_sentiment", ascending=True).head(10)

# Seleciona colunas desejadas
colunas = ["text_content_anonymous", "score_sentiment"]
if "score_misinformation" in df.columns:
    colunas.append("score_misinformation")

# Junta e exibe
amostras = pd.concat([top_positivas[colunas], top_negativas[colunas]])
print(amostras.to_string(index=False))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nasci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\nasci\AppData\Local\Temp\ipykernel_18196\3602231730.py:13: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dataset_zap_2.csv")  # ou dataset_zap_2.csv


In [2]:
amostras.to_csv("amostras_polaridade.csv", index=False, encoding='utf-8-sig')